# ✍️ Exercise: Intro to MLFlow - Part III

Now that we have loged models into MLFlow it's time to learn how register them and deploy them to a production environment.


- Load a regression dataset
- Train a model
- Log the model into MLFlow
- Register the model
- Stage the model into production/development
- Deploy the model using MLFlow

In [5]:
from sklearn import datasets


# Download dataset and convert to pandas dataframe
housing_dataset = datasets.fetch_california_housing()
X = housing_dataset.data
y = housing_dataset.target

## Exercise I: Split the Data into Train and Test Sets

💡 Remember that we need to split our data into train and test sets. We can use the [`train_test_split` function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from `sklearn.model_selection` to do this. We should store the split into `X_train`, `y_train`, `X_test`, `y_test`.

In [6]:
from sklearn.model_selection import train_test_split


RANDOM_STATE = 42
TEST_SIZE = 0.2

# 👇 Add the relevant code below to split the data into training and testing sets


## Exercise II: Train a Linear Regression Model

Then, train a [**linear regression model** using the scikit-learn library](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html).

1. 👉 Initialize the model calling the `LinearRegression` class.
2. 👉 Train the model using the `fit` method.

In [7]:
from sklearn.linear_model import LinearRegression


# Add code to train the model 👇


LinearRegression()

## Exercise III: Compute the Accuracy of the Model

Finally, compute the accuracy of the model using the [`mean_squared_error` function](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) from the `sklearn.metrics` module.

1. 👉 Compute the predictions by passing the `X_test` to the `predict` method of the model.
2. 👉 Compute the accuracy using the `mean_squared_error` function and passing the `y_test` and the `predictions` as arguments.
3. 👉 Print the accuracy.

In [11]:
from sklearn.metrics import mean_squared_error


# Add code to calculate the mean squared error 👇


## Exercise IV: Create a Run and log the model and metrics.

1. 👉 Log the mean squared error metric using `mlflow.log_metric` function
2. 👉 Log the model using the `mlflow.sklearn.log_model` function.

In [18]:
import mlflow


EXPERIMENT_NAME = "intro-to-mlflow"
MLFLOW_TRACKING_URI = "http://localhost:5000"

# connect to MLFlow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# Get the experiment
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

# launch a run to log the model
with mlflow.start_run(
    experiment_id=experiment.experiment_id,
) as run:
    
    # Add code to log the model, and the mean squared error 👇

/home/sngular/.pyenv/versions/3.11.5/envs/mlops-course/lib/python3.11/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/sngular/.pyenv/versions/3.11.5/envs/mlops-course/lib/python3.11/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## Exercise V: Register the model

Registering a model in MLFlow is a way to keep track of the different versions of the same model. Registered models have different versions that track changes in the model and allows

1. 👉 Get the **run ID** of the model you want to register using `run.info.run_id`.
2. 👉 Register the model using the `mlflow.register_model` function.

In [19]:
# register the model for this run
MODEL_NAME = "housing-price-predictions"  # change this to your model name


# Compute model path: models stored in a run follow this convention
model_path = f"runs:/{run_id}/model"  # fill the `run_id`` variable

Successfully registered model 'housing-price-predictions'.
2024/02/29 10:24:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: housing-price-predictions, version 1


✅ Registered model version: 1!


Created version '1' of model 'housing-price-predictions'.


## Exercise VI: Deploy a model

Deploying a model is a complex task that involves many steps. MLFlow simplifies this process by providing a set of tools to deploy models to different platforms. In this exercise, we will deploy a model to a local server. 

First, you need to connect the terminal to the MLFlow Server by setting the `MLFLOW_TRACKING_URI` environment variable. 

```bash
export MLFLOW_TRACKING_URI=http://localhost:5000
```

Then, you can deploy the model using the `mlflow models serve` command **in your terminal**:

```bash
mlflow models serve --model-uri models:/<model_name>/<model_version> --port 5001
```

Where `<model_name>` is the name of the model and `<model_version>` is the version of the model you want to deploy. You can find the name and version of the model in the MLFlow UI. Also the `--port` argument is the port where the server will be running. It's important to choose a port different than the `5000` port where the MLFlow server is running.

## BONUS: Make a request to the model

Finally, make a request to the model using the `requests` library. You can use the following code to make a request to the model:

In [20]:
import requests
import json
import numpy as np

# Define the URL and payload (JSON data)
url = 'http://localhost:5001/invocations'
headers = {'Content-Type': 'application/json'}

# Create a list representing the (100, 1) vector
input_vector = np.random.rand(2, 8).tolist()

# Create a dictionary with the 'inputs' key and the input_vector
payload = {'inputs': input_vector}

# Convert the payload to JSON format
json_payload = json.dumps(payload)

# Make a POST request
response = requests.post(url, headers=headers, data=json_payload)

# Check the response
if response.status_code == 200:
    print("Request successful. Response:")
    print(response.text)
else:
    print(f"Request failed with status code {response.status_code}")
    print(response.text)

Request successful. Response:
{"predictions": [-36.83689486654794, -37.10261875122955]}
